# Qwen Coder - Advanced Version

Full-featured coding assistant with smart tool handling.

In [ ]:
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!pip install -q flask requests
print('✅ Setup done')

In [ ]:
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
!ollama pull qwen2.5-coder:7b
print('\n✅ Model ready')

In [ ]:
from flask import Flask, request, jsonify
import requests as req
import json, time, uuid, threading, re

app = Flask(__name__)

SYSTEM_PROMPT = '''You are an elite AI coding assistant with deep expertise across all programming languages, frameworks, and development paradigms.

## Core Principles
1. **Precision**: Write clean, efficient, production-ready code
2. **Best Practices**: Follow industry standards, design patterns, and security principles
3. **Clarity**: Explain complex concepts simply when asked
4. **Efficiency**: Solve problems with minimal, elegant solutions

## Response Guidelines
- For **greetings/questions**: Respond conversationally - do NOT use tools
- For **code requests**: Write complete, working code with proper error handling
- For **file operations**: Use the appropriate tool (write, edit, read)
- For **debugging**: Analyze systematically, identify root causes
- For **architecture**: Consider scalability, maintainability, security

## Code Style
- Use meaningful variable/function names
- Add concise comments for complex logic
- Handle edge cases and errors gracefully
- Follow language-specific conventions (PEP8 for Python, ESLint for JS, etc)

## Tool Usage Rules
- ONLY use tools when the user explicitly requests an action (create, edit, read, run, search)
- For simple conversations or code explanations, respond with text only
- When using tools, provide the exact required arguments
- Never invent tool names - only use tools from the provided list

## Expertise Areas
- Full-stack development (React, Node, Python, Go, Rust)
- DevOps & Cloud (Docker, K8s, AWS, GCP, Azure)
- Databases (SQL, NoSQL, Graph, Vector)
- AI/ML (PyTorch, TensorFlow, LangChain, RAG)
- Security (OWASP, encryption, auth patterns)
- Performance optimization & debugging

Be concise, accurate, and helpful. Focus on solving the user\'s problem efficiently.'''

def needs_tools(user_msg):
    msg = user_msg.lower()
    # Action keywords that indicate tool usage
    action_kw = ['create', 'write', 'make', 'generate', 'add', 'new',
                 'edit', 'modify', 'update', 'change', 'fix', 'refactor',
                 'delete', 'remove', 'read', 'open', 'show', 'view', 'list',
                 'run', 'execute', 'test', 'build', 'deploy',
                 'search', 'find', 'grep', 'locate',
                 'file', 'folder', 'directory', 'project',
                 'todo', 'task', 'bash', 'terminal', 'command', 'script']
    return any(kw in msg for kw in action_kw)

def get_tool_names(tools):
    names = set()
    for t in tools or []:
        if t.get('type') == 'function' and 'function' in t:
            names.add(t['function'].get('name', ''))
    return names

def extract_tool_call(text, valid_tools):
    if not text or not valid_tools:
        return None
    
    # Code blocks first
    code_block = re.search(r'```(?:json)?\s*([\s\S]*?)```', text)
    if code_block:
        try:
            data = json.loads(code_block.group(1).strip())
            name = data.get('name', '')
            if name in valid_tools:
                return name, json.dumps(data.get('arguments', {}))
        except:
            pass
    
    # Inline JSON
    match = re.search(r'\{[\s\S]*?"name"\s*:\s*"([^"]+)"[\s\S]*?"arguments"\s*:\s*(\{[^{}]*\})', text)
    if match and match.group(1) in valid_tools:
        return match.group(1), match.group(2)
    
    return None

@app.route('/v1/models', methods=['GET'])
def models():
    return jsonify({"object": "list", "data": [{"id": "qwen2.5-coder:7b", "object": "model", "owned_by": "ollama"}]})

@app.route('/v1/chat/completions', methods=['POST'])
def chat():
    data = request.json
    messages = data.get('messages', [])
    tools = data.get('tools', [])
    valid_tools = get_tool_names(tools)
    
    # Get last user message
    last_user_msg = ''
    for m in reversed(messages):
        if m.get('role') == 'user' and m.get('content'):
            last_user_msg = str(m.get('content', ''))
            break
    
    use_tools = needs_tools(last_user_msg)
    print(f"[{time.strftime('%H:%M:%S')}] '{last_user_msg[:40]}...' tools={use_tools}")
    
    # Build messages with system prompt
    modified_messages = [{'role': 'system', 'content': SYSTEM_PROMPT}]
    for m in messages:
        if m.get('role') != 'system':
            modified_messages.append(m)
    
    # Call Ollama
    try:
        payload = {
            'model': 'qwen2.5-coder:7b',
            'messages': modified_messages,
            'stream': False,
            'options': {'num_ctx': 16384}
        }
        if use_tools and tools:
            payload['tools'] = tools
        
        r = req.post('http://localhost:11434/v1/chat/completions', json=payload, timeout=180)
        result = r.json()
        msg = result.get('choices', [{}])[0].get('message', {})
        content = msg.get('content', '')
        tool_calls = msg.get('tool_calls', [])
        
    except Exception as e:
        print(f"Error: {e}")
        content = "I'm here to help! What would you like to work on today?"
        tool_calls = []
    
    # Ollama returned tool calls
    if tool_calls:
        print(f"  → Tool calls from Ollama")
        return jsonify({
            "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
            "object": "chat.completion",
            "created": int(time.time()),
            "model": "qwen2.5-coder:7b",
            "choices": [{"index": 0, "message": {"role": "assistant", "content": None, "tool_calls": tool_calls}, "finish_reason": "tool_calls"}],
            "usage": {"prompt_tokens": 100, "completion_tokens": 100, "total_tokens": 200}
        })
    
    # Parse tool calls from text if needed
    if use_tools and valid_tools and content:
        tool_result = extract_tool_call(content, valid_tools)
        if tool_result:
            name, args = tool_result
            print(f"  → Parsed tool: {name}")
            return jsonify({
                "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
                "object": "chat.completion",
                "created": int(time.time()),
                "model": "qwen2.5-coder:7b",
                "choices": [{
                    "index": 0,
                    "message": {
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [{
                            "id": f"call_{uuid.uuid4().hex[:8]}",
                            "type": "function",
                            "function": {"name": name, "arguments": args}
                        }]
                    },
                    "finish_reason": "tool_calls"
                }],
                "usage": {"prompt_tokens": 100, "completion_tokens": 100, "total_tokens": 200}
            })
    
    # Clean up and return text
    if content:
        content = re.sub(r'```json[\s\S]*?```', '', content).strip()
        content = re.sub(r'\{\s*"name"[^}]+\}', '', content).strip()
    if not content:
        content = "Hello! I'm your coding assistant. How can I help you today?"
    
    print(f"  → Text response ({len(content)} chars)")
    return jsonify({
        "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
        "object": "chat.completion",
        "created": int(time.time()),
        "model": "qwen2.5-coder:7b",
        "choices": [{"index": 0, "message": {"role": "assistant", "content": content}, "finish_reason": "stop"}],
        "usage": {"prompt_tokens": 100, "completion_tokens": len(content.split()), "total_tokens": 100 + len(content.split())}
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "healthy", "model": "qwen2.5-coder:7b"})

threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
time.sleep(2)
print('✅ Advanced API running on port 5000!')

In [ ]:
# Test
import requests

tests = [
    ("Hi", False),
    ("What is Python?", False),
    ("Create a file test.py", True),
    ("Write a sorting algorithm", True),
]

for msg, expect_tools in tests:
    r = requests.post('http://localhost:5000/v1/chat/completions', json={
        'model': 'qwen2.5-coder:7b',
        'messages': [{'role': 'user', 'content': msg}],
        'tools': [{'type': 'function', 'function': {'name': 'write'}}]
    })
    has_tools = r.json()['choices'][0].get('message', {}).get('tool_calls') is not None
    status = '✅' if has_tools == expect_tools else '⚠️'
    print(f"{status} '{msg}' -> tools={has_tools} (expected {expect_tools})")

In [ ]:
import subprocess, re
from IPython.display import display, HTML

tunnel = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''<div style="background:linear-gradient(135deg,#667eea,#764ba2);padding:30px;border-radius:20px">
            <h2 style="color:white;margin:0">🚀 Elite Coding Assistant Ready</h2>
            <p style="color:white;font-size:20px;font-family:monospace;margin:15px 0">{url}/v1</p>
            <p style="color:#ddd;margin:0">Advanced prompt • Smart tool detection • Production-ready</p>
            </div>'''))
            break

for line in tunnel.stdout:
    print(line, end='')